In [75]:
import glob
MASS_dir = sorted(glob.glob('/Users/josangmin/EEG/MASS_SS1/**'))

In [76]:
len(MASS_dir)

19

In [81]:
from torch.utils.data import Dataset
import torch
import pickle
import mne
import numpy as np
import time


class EEGLoader(Dataset):
    def __init__(self, files):
        self.files = files
        self.sequence_length = 200
        
        # sample을 split해줬을 때 몇개로 split되는지 누적해서 저장, i번째 data를 찾을 때 data_adress 각 값이 기준이 됨
        data_adress = [0]
        ad = 0
        for i in range(len(self.files)):
          with open(file = files[i] , mode='rb') as f:
            sample = pickle.load(f)
          c,t = sample.shape
          t = int(c*t*2/self.sequence_length)
          ad += t
          
          data_adress.append(ad)
        print(data_adress[:100])
        self.data_adress = data_adress


    def preprocessing_sleepedf(self,data):
        data = mne.filter.resample(data, up = 2.0) # upsampling to 200Hz  
        return data

    def preprocessing_MASS(self,data):
        data = mne.filter.resample(data, down = 1.28) # downsampling to 200Hz  
        return data

    def split_data(self, data):
        L = self.sequence_length
        channels, length = data.shape
        a = L*int(length/L)
        
        if length == a:
          data = np.reshape(data,(int(length/L*channels),1,L))
        
        else:
          data = data[:,:a]
          data = np.reshape(data,(int(a/L*channels),1,L))
        data = np.squeeze(data,1)
        return data
    
    # def get_index(self):  
    #     data_adress = []
    #     for i in range(len(self.files)):
    #       sample = pickle.load(open(self.files[i]))
    #       _,t = sample.shape
    #       ad = int(t*2/self.sequence_length)*self.sequence_length
    #       data_adress.append(ad)
    #     return data_adress

    def __getitem__(self, index):
#         start_T = time.time()
          for i in range(len(self.data_adress)):
            if index < self.data_adress[i]:
              break
          
          with open(file = self.files[i-1] , mode='rb') as f:
            sample = pickle.load(f)
          
          
          sample1 = self.preprocessing_sleepedf(sample)
          split_sample = self.split_data(sample1)
          
#         end_T = time.time()
#         print('time:', end_T - start_T)
          return split_sample[index - self.data_adress[i-1],:]
          
    def __len__(self):
        return self.data_adress[-1]


In [86]:
MASS_dir = MASS_dir[:5]

In [88]:
from torch.utils.data import DataLoader

trainLoader = DataLoader(EEGLoader(MASS_dir), batch_size = 1, shuffle=True)

[0, 1526353, 3106804, 4530952, 6099330, 7626236]


In [90]:
import time
start_T = time.time()
for batch_idx, batch in enumerate(trainLoader):
  print('batch_idx:',batch_idx,' ',batch.shape)
  end_T = time.time()
  print('time:', end_T - start_T)
  if batch_idx >= 2:
        break

batch_idx: 0   torch.Size([1, 200])
time: 140.19377517700195
batch_idx: 1   torch.Size([1, 200])
time: 244.10474014282227
batch_idx: 2   torch.Size([1, 200])
time: 362.50570607185364


Sleep data의 경우는 한 smaple당 데이터가 너무 커서 load,split, preprocessing할 때 너무 오래 걸림.
그래도 원하는 사이즈의 인풋으로 받을 수 있음.
Sleep data는 30초씩 잘라서 저장할 필요.

------

sleep edf 30초씩 잘라서 저장한 파일

In [92]:
import glob 
sleepedf_list = glob.glob('/Users/josangmin/EEG/Sleep_edfx_30s/**')

In [97]:
print(len(sleepedf_list))

416272


In [95]:
from torch.utils.data import DataLoader

trainLoader = DataLoader(EEGLoader(sleepedf_list), batch_size = 256, shuffle=True)

[0, 60, 120, 180, 240, 300, 360, 420, 480, 540, 600, 660, 720, 780, 840, 900, 960, 1020, 1080, 1140, 1200, 1260, 1320, 1380, 1440, 1500, 1560, 1620, 1680, 1740, 1800, 1860, 1920, 1980, 2040, 2100, 2160, 2220, 2280, 2340, 2400, 2460, 2520, 2580, 2640, 2700, 2760, 2820, 2880, 2940, 3000, 3060, 3120, 3180, 3240, 3300, 3360, 3420, 3480, 3540, 3600, 3660, 3720, 3780, 3840, 3900, 3960, 4020, 4080, 4140, 4200, 4260, 4320, 4380, 4440, 4500, 4560, 4620, 4680, 4740, 4800, 4860, 4920, 4980, 5040, 5100, 5160, 5220, 5280, 5340, 5400, 5460, 5520, 5580, 5640, 5700, 5760, 5820, 5880, 5940]


In [96]:
import time
start_T = time.time()
for batch_idx, batch in enumerate(trainLoader):
  print('batch_idx:',batch_idx,' ',batch.shape)
  end_T = time.time()
  print('time:', end_T - start_T)
  if batch_idx >= 10:
        break

batch_idx: 0   torch.Size([256, 200])
time: 8.319709300994873
batch_idx: 1   torch.Size([256, 200])
time: 12.694368124008179
batch_idx: 2   torch.Size([256, 200])
time: 17.293900966644287
batch_idx: 3   torch.Size([256, 200])
time: 22.151576042175293
batch_idx: 4   torch.Size([256, 200])
time: 26.6968891620636
batch_idx: 5   torch.Size([256, 200])
time: 31.25409507751465
batch_idx: 6   torch.Size([256, 200])
time: 35.35506200790405
batch_idx: 7   torch.Size([256, 200])
time: 39.457611083984375
batch_idx: 8   torch.Size([256, 200])
time: 43.430830001831055
batch_idx: 9   torch.Size([256, 200])
time: 47.56143832206726
batch_idx: 10   torch.Size([256, 200])
time: 51.451027154922485


sleep edf 6개 sample 올려도 메모리 터지는 문제,
416272개가 153개 sample을 30초 씩 잘라서 저장한 건데, 그것의 절반인 20만개 올려도 메모리 터지지 않음.